# Compute the NLL metric before and after tracking

In [1]:
import numpy as np
import torch
import math
import os
import properscoring as ps
import warnings
warnings.filterwarnings('ignore')

In [2]:
before_path = "../det/check/check_loss_two_step_center_rh_ind/"
after_path = "./byte_tracker/cov/"
after_cfk_path = after_path + "cfk/"
after_nlla_path = after_path + "nlla/"
after_origin_path = after_path + "origin/"
after_cfk_nlla_path = after_path + "cfk_nlla/"
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/test_scenes.txt"
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885]}

In [3]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,r,h] where x,y is the centre of the box and r is
      the aspect ratio
    """
    w = bbox[2]
    h = bbox[3]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, r, h])

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = torch.from_numpy(convert_bboxs_to_zs(matched_dets[:,:4]))
    target = torch.from_numpy(convert_bboxs_to_zs(matched_gts[:,:4]))
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    cov = torch.from_numpy(cov)
    std = torch.sqrt(cov)
    predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
    negative_log_prob = - predicted_normal_dists.log_prob(target)
    negative_log_prob = torch.clamp(negative_log_prob, min = 0)
    negative_log_prob = torch.sum(negative_log_prob,axis=1)/4
    # if sum(negative_log_prob) > 10*7:
    #     print(dets)
    #     print(gts)
    #     print(matched_indices)
    #     print(negative_log_prob)
    #     print(iou_matrix)
    #     print(matched_dets)
    #     print(matched_gts)
    #     return [], []
    return negative_log_prob.tolist(), matched_indices

def compute_nll_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:

            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_idx >= len(det_datas):
                    break
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["upperbound", "disco","lowerbound"]
    for mode in mode_list:
        nll = compute_nll_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

## before tracking

In [4]:
mode = "upperbound"
nll = compute_nll_for_mode(mode, before_path, True, 0.5, True)
print(f"NLL for {mode} is {nll}")

NLL for upperbound is 53.78922865018845


In [5]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 53.78922865018845
NLL for disco is 71.58428929992485
NLL for lowerbound is 84.5428391808751


In [6]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 3204.5893719730598
NLL for disco is 5780.966739348752
NLL for lowerbound is 5395.272243765268


In [7]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 31.800183472251298
NLL for disco is 46.725078928276304
NLL for lowerbound is 21.838259327222616


In [8]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1966.6107415645718
NLL for disco is 3856.4190231386438
NLL for lowerbound is 1862.2037339378617


In [9]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10]}
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 319.3816603609519
NLL for disco is 577.0204889917645
NLL for lowerbound is 538.4593564283585


In [10]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 195.58397234090145
NLL for disco is 384.5700219988893
NLL for lowerbound is 185.1610723820612


## after tracking origin

In [11]:
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885]}
show_one_result(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 20.95257657137378
NLL for disco is 116.95265533161864
NLL for lowerbound is 7.21952550390977


In [12]:
show_one_result(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 18.599546348665335
NLL for disco is 6.339943740304376
NLL for lowerbound is 3.2475714091450554


## after tracking cfk

In [13]:
show_one_result(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 31.506232666450455
NLL for disco is 52.427947939989025
NLL for lowerbound is 55.52599474807452


In [14]:
show_one_result(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 23.79823602326444
NLL for disco is 42.120769782372626
NLL for lowerbound is 22.40044172498877


## after tracking nlla

In [15]:
show_one_result(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 2159.373751681863
NLL for disco is 123.69860360995361
NLL for lowerbound is 8.196590759313471


In [16]:
show_one_result(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 2207.47225628959
NLL for disco is 7.060356317380336
NLL for lowerbound is 3.544019148748239


## after tracking cfk + nlla

In [17]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 36.95258754497168
NLL for disco is 54.5004854636926
NLL for lowerbound is 69.87568676883734


In [18]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 29.280835449935427
NLL for disco is 43.5364815440318
NLL for lowerbound is 23.500359895091695


In [19]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 0.0065616422113331276, 9.810218760278726], "disco": [1.5844876978208504, 16.85182149486315, 0.007767244933272351, 10.737039123121171], "lowerbound": [0.9496513748504812, 14.405661414225111, 0.0015879443245170666, 6.56055795747982]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 6.470488639693675
NLL for disco is 8.120728241573676
NLL for lowerbound is 15.204318345104362


In [20]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 3.851571320150271
NLL for disco is 6.009559973197374
NLL for lowerbound is 3.9345103436779953


In [21]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 0.2799106202333024, 23.82625371377505], "disco": [5.134614410746115, 38.66801996668446, 0.3031539483223637, 22.899039617727098], "lowerbound": [4.463060508876752, 30.744574219188294, 0.028465764379049255, 18.922759296842013]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 2.359550175913343
NLL for disco is 3.146641854757538
NLL for lowerbound is 4.244379136884319


In [22]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 1.6946910584173314
NLL for disco is 2.4313519337806815
NLL for lowerbound is 1.521384462568064


# CRPS

In [23]:
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885]}
def compute_crps(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])
    target = convert_bboxs_to_zs(matched_gts[:,:4])
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    std = np.sqrt(cov)
    crps = ps.crps_gaussian(target, 
                               pred,
                               std)
    return np.mean(crps, axis=1).tolist(), matched_indices

def compute_crps_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_idx >= len(det_datas):
                    break
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_crps(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result_crps(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        nll = compute_crps_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

In [24]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.6210594718664262
NLL for upperbound is 0.5816432159196336
NLL for lowerbound is 0.8500387495700458


In [25]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.5872349092269485
NLL for upperbound is 0.5261312513418207
NLL for lowerbound is 0.5962767555784172


In [26]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 0.5818908855768533
NLL for upperbound is 0.5470448428670751
NLL for lowerbound is 0.818631608889578


In [27]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for disco is 0.5486457074709636
NLL for upperbound is 0.4921802293026843
NLL for lowerbound is 0.5680183662946537


In [28]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.5085656474847818
NLL for upperbound is 0.45023065374726584
NLL for lowerbound is 0.6103605574869051


In [29]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.5007673850051751
NLL for upperbound is 0.42705552589972573
NLL for lowerbound is 0.5075522524466236


In [31]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.48919256897024194
NLL for upperbound is 0.4337474953766887
NLL for lowerbound is 0.5770330428863654


In [30]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.47742055541588785
NLL for upperbound is 0.40235982198941517
NLL for lowerbound is 0.4753167511424003


In [32]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.498483370966158
NLL for upperbound is 0.4355359659734821
NLL for lowerbound is 0.5920916526088508


In [33]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.4914778587578079
NLL for upperbound is 0.413135260823531
NLL for lowerbound is 0.49745348372266635


In [35]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.5006122925584815
NLL for upperbound is 0.4472146751819923
NLL for lowerbound is 0.5939409244198556


In [36]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.48493438763049174
NLL for upperbound is 0.4157518470589673
NLL for lowerbound is 0.48516689905432936


In [37]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 0.0065616422113331276, 9.810218760278726], "disco": [1.5844876978208504, 16.85182149486315, 0.007767244933272351, 10.737039123121171], "lowerbound": [0.9496513748504812, 14.405661414225111, 0.0015879443245170666, 6.56055795747982]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 0.4580827673186847
NLL for upperbound is 0.40071154945244064
NLL for lowerbound is 0.5549854906185028


In [38]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for disco is 0.4501592770066613
NLL for upperbound is 0.3776944155138197
NLL for lowerbound is 0.4525772872676113


In [39]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 0.2799106202333024, 23.82625371377505], "disco": [5.134614410746115, 38.66801996668446, 0.3031539483223637, 22.899039617727098], "lowerbound": [4.463060508876752, 30.744574219188294, 0.028465764379049255, 18.922759296842013]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 0.4455924195154952
NLL for upperbound is 0.3957224363892877
NLL for lowerbound is 0.538560348535857


In [40]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for disco is 0.4376381150073242
NLL for upperbound is 0.37269378287963745
NLL for lowerbound is 0.4378584352336107


In [41]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10]}
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 0.6042838523378714
NLL for upperbound is 0.5642151344735987
NLL for lowerbound is 0.8325363548932871


In [42]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for disco is 0.5706108380883314
NLL for upperbound is 0.5089887626024044
NLL for lowerbound is 0.5792202925021502
